`Author: Victor Radermecker | Date: 11/10/2021 | Project: Master Thesis`

This notebook computes the final demand estimation using the PPR and INDR ratios.

In [1]:
import json
import math
import os

# Importing local libraries
import sys

import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import *
from tqdm import tqdm

sys.path.insert(0, "../src")
from BrusselsMap import BrusselsMap
from GeoJsonHandler import GeoJsonHandler

pd.set_option("display.max_rows", 500)

In [2]:
%load_ext autoreload
%autoreload 2

## Demand Estimation Computation

#### Proximus data

In [9]:
prox = pd.read_csv("../data/proximus/proximus_aggregated.csv")
prox.head()

,NAME_FRE,REGULAR,IRREGULAR
0,ALTITUDE 100,741855,674969
1,ANDERLECHT CENTRE - WAYEZ,696621,642094
2,ANNEESSENS,696328,663316
3,AUDERGHEM CENTRE,718066,719010
4,AVENUE LEOPOLD III,696597,638930


#### Convert to Energy 

In [ ]:
dists = pd.read_csv("../data/final_data/DemandClusters-ODMatrix-Km.csv").set_index(
    "origin"
)

# Convert number of trips into distance
dists_long = (
    dists.unstack()
    .reset_index()
    .rename(
        columns={"level_0": "destinationNB", "origin": "originNB", 0: "travelDistance"}
    )[["originNB", "destinationNB", "travelDistance"]]
)
prox = pd.merge(prox, dists_long, on=["originNB","destinationNB"], how="outer").fillna(0)

# Convert to Energy using IEA Assumption on EV's consumption (0.22kWh/km)
prox['regularDemand'] = prox['regularTripSample'] * prox['travelDistance']
prox['irregularDemand'] = prox['irregularTripSample'] * prox['travelDistance']
prox['totalDemand'] = prox['regularDemand'] + prox['irregularDemand']

prox['regularEnergyDemand'] = prox['regularDemand'] * 0.22 
prox['irregularEnergyDemand'] = prox['irregularDemand'] * 0.22 

#Aggregate data and save
prox = prox.groupby(by=['destinationNB'])[['regularTripSample','irregularTripSample', 'travelDistance', 'regularDemand', 'irregularDemand','totalDemand', 'regularEnergyDemand','irregularEnergyDemand']].sum().reset_index()
prox.rename(columns={'destinationNB':'NAME_FRE'}, inplace=True)
prox.to_csv('../data/final_data/DemandEnergy.csv', index=False)

#### PPR Ratio

In [11]:
# Loading parking and population data
parking = pd.read_csv("../data/final_data/parkings_ratio.csv")
parking.head()

,NAME_FRE,AREA,DENS,POP,AVG_HOUS,HOUSEHOLDS,NOT REGLEMENTED,RESERVED,REGLEMENTED,PRIVATE,PRIVATE/HOUS
0,CONSCIENCE,0.469490,17518.92,8225,2.33,3530,0,57,1260,551,0.156091
1,HELMET,0.718634,17717.47,12732,2.54,5013,0,81,2315,552,0.110114
2,VIEUX LAEKEN OUEST,0.499646,16781.39,8385,2.55,3288,0,90,996,310,0.094282
3,VIEUX LAEKEN EST,1.040450,18272.53,19012,2.61,7284,0,211,2858,664,0.091159
4,INDUSTRIE NORD,6.733601,261.99,1764,2.18,809,1277,26,248,218,0.269468


#### INDR Ratio

### Merge datasets together

In [ ]:

prox = pd.merge(prox, data[['NAME_FRE','PRIVATE/HOUS']], on="NAME_FRE", how="outer")
prox = prox.fillna(0)

prox['REGULAR_CORR'] = prox['REGULAR'] * (1-prox['PRIVATE/HOUS'])
prox['IRREGULAR_CORR'] = prox['IRREGULAR'] * (1-prox['PRIVATE/HOUS'])
prox.head()

In [ ]:

#Load geodata
r_path = r"../data/final_data/sector.json"
path = os.path.join(os.getcwd(), r_path)
sectors = GeoJsonHandler(path, "NAME_FRE")

#### INDR Ratio Data